In [1]:
from autoprotocol import Container, WellGroup, Protocol, Unit
from autoprotocol.instruction import *
import json

In [2]:
class ContainerGroup:
    def __init__(self,name, containers, timing_type="sync"):
        self.name = name
        self.containers = containers
        if timing_type in ["sync", "async"]:  
            self.timing_type = timing_type
        else:
            raise TypeError("Timing type must be either \"async\" or \"sync\"")
        
        for container in self.containers:
            try:
                all_wells_group.extend(container.all_wells())
            except NameError:
                all_wells_group = container.all_wells()
        self.wells = all_wells_group
            
    def __len__(self):
        """
        Return the number of Containers in a ContainerGroup.
        """
        return len(self.containers)

In [3]:
def new_incubate(self, ref, where, duration, shaking=False, co2=0,
         uncovered=False):
    """
    Move plate or plates to designated thermoisolater or ambient area for incubation
    for specified duration.
    
    If the ref supplied is of type ContainerGroup with a timing_type of "sync" then
    identical incubate instructions will be generated for each Container in the
    ContainerGroup to be executed as close as physically possible to synchronous.
    
    If the timing_type of the ContainerGroup is "async" then just a series of identical
    incubate instructions will be generated for each Container.
    
    Ignore example text.
    Example Usage:
    .. code-block:: python
        p = Protocol()
        sample_plate = p.ref("sample_plate",
                             None,
                             "96-pcr",
                             storage="warm_37")
        # a plate must be sealed/covered before it can be incubated
        p.seal(sample_plate)
        p.incubate(sample_plate, "warm_37", "1:hour", shaking=True)
    Autoprotocol Output:
    .. code-block:: json
        "instructions": [
            {
              "object": "sample_plate",
              "op": "seal"
            },
            {
              "duration": "1:hour",
              "where": "warm_37",
              "object": "sample_plate",
              "shaking": true,
              "op": "incubate",
              "co2_percent": 0
            }
          ]
    """    
    if not isinstance(ref, (Container, ContainerGroup)):
        raise TypeError("Ref needs to be of type Container or ContainerGroup")
    allowed_uncovered = ["ambient"]
    
    
    if isinstance(ref, ContainerGroup):
        for container in ref.containers:
            if uncovered and (where not in allowed_uncovered or shaking):
                raise RuntimeError("If incubating uncovered, "
                           "location must be in {} and not "
                           "shaking.".format(', '.join(allowed_uncovered)))
            if not uncovered:
                self._add_cover(container, "incubate")
        if ref.timing_type == "sync":
            time_points = []
            for container in ref.containers:
                self.instructions.append(Incubate(container, where, duration, shaking, co2))
                time_points.append(self.get_instruction_index())
            p.add_time_constraint(
                {"mark": time_points[0], "state": "start"},
                {"mark": time_points[-1], "state": "start"},
                Unit(len(ref),"minute"), True)
        else:
            for container in ref.containers:
                self.instructions.append(Incubate(container, where, duration, shaking, co2))
    else:
        if uncovered and (where not in allowed_uncovered or shaking):
            raise RuntimeError("If incubating uncovered, "
                "location must be in {} and not "
                "shaking.".format(', '.join(allowed_uncovered)))
        if not uncovered:
            self._add_cover(ref, "incubate")
        self.instructions.append(Incubate(ref, where, duration, shaking, co2))
        
Protocol.incubate = new_incubate

In [4]:
p = Protocol()

container1 = p.ref("one", cont_type = "96-flat",storage="cold_4", discard=False)
container2 = p.ref("two", cont_type = "384-flat",storage="cold_4", discard=False)
container3 = p.ref("thr", cont_type = "96-flat",storage="cold_4", discard=False)

# Create a synchronous container group
cg = ContainerGroup("cg", [container1,container2,container3], "sync")

print len(cg.wells)
cg.wells.set_volume("50:microliter")

576


WellGroup([Well(Container(one), 0, 50.0:microliter), Well(Container(one), 1, 50.0:microliter), Well(Container(one), 2, 50.0:microliter), Well(Container(one), 3, 50.0:microliter), Well(Container(one), 4, 50.0:microliter), Well(Container(one), 5, 50.0:microliter), Well(Container(one), 6, 50.0:microliter), Well(Container(one), 7, 50.0:microliter), Well(Container(one), 8, 50.0:microliter), Well(Container(one), 9, 50.0:microliter), Well(Container(one), 10, 50.0:microliter), Well(Container(one), 11, 50.0:microliter), Well(Container(one), 12, 50.0:microliter), Well(Container(one), 13, 50.0:microliter), Well(Container(one), 14, 50.0:microliter), Well(Container(one), 15, 50.0:microliter), Well(Container(one), 16, 50.0:microliter), Well(Container(one), 17, 50.0:microliter), Well(Container(one), 18, 50.0:microliter), Well(Container(one), 19, 50.0:microliter), Well(Container(one), 20, 50.0:microliter), Well(Container(one), 21, 50.0:microliter), Well(Container(one), 22, 50.0:microliter), Well(Conta

In [5]:
p.incubate(cg, "warm_37", "50:minute", False, 0)

In [6]:
print json.dumps(p.as_dict(), indent=2)

{
  "refs": {
    "one": {
      "new": "96-flat", 
      "store": {
        "where": "cold_4"
      }
    }, 
    "thr": {
      "new": "96-flat", 
      "store": {
        "where": "cold_4"
      }
    }, 
    "two": {
      "new": "384-flat", 
      "store": {
        "where": "cold_4"
      }
    }
  }, 
  "time_constraints": [
    {
      "to": {
        "instruction_start": 5
      }, 
      "less_than": "3.0:minute", 
      "from": {
        "instruction_start": 3
      }
    }, 
    {
      "to": {
        "instruction_start": 3
      }, 
      "less_than": "3.0:minute", 
      "from": {
        "instruction_start": 5
      }
    }
  ], 
  "instructions": [
    {
      "lid": "standard", 
      "object": "one", 
      "op": "cover"
    }, 
    {
      "lid": "standard", 
      "object": "two", 
      "op": "cover"
    }, 
    {
      "lid": "standard", 
      "object": "thr", 
      "op": "cover"
    }, 
    {
      "where": "warm_37", 
      "object": "one", 
      "co2_percen